# 📋 Utvärdering & Verifiering - Automatiska checks och kvittogenerering

**Mål**: Förstå hur automatisk verifiering fungerar och vad som kontrolleras.

I detta notebook kommer vi att:
- Förstå vad utvärdering innebär (confusion matrix, accuracy)
- Se hur automatisk verifiering fungerar
- Analysera kvittot och dess komponenter
- Reflektera över vad som saknas för "produktion"

> **💡 Tips**: Automatisk verifiering är kritiskt för att säkerställa att lektionen fungerar korrekt!


## 🤔 Vad är utvärdering och varför behöver vi det?

**Utvärdering** = testa modellen på data den inte har sett under träning.

**Vad vi mäter**:
- **Accuracy** - hur många förutsägelser som är rätta
- **Confusion matrix** - detaljerad breakdown av rätta/felaktiga förutsägelser
- **Per-class performance** - hur bra modellen är på varje klass

**Varför viktigt**:
- **Validering** - säkerställer att modellen faktiskt fungerar
- **Debugging** - visar vilka klasser som är svåra
- **Jämförelse** - kan jämföra olika modeller/inställningar

<details>
<summary>🔍 Klicka för att se vad en confusion matrix visar</summary>

**Confusion matrix**:
- **Diagonal** = rätta förutsägelser
- **Off-diagonal** = felaktiga förutsägelser
- **Per class** = precision, recall för varje klass

</details>


In [ ]:
# Kör utvärdering på vår modell
print("🔍 Kör utvärdering...")

# Använd modellen från föregående notebooks (eller skapa en snabb)
!python -m piedge_edukit.train --fakedata --no-pretrained --epochs 1 --batch-size 256 --output-dir ./models_eval


In [ ]:
# Kör utvärdering med begränsat antal samples (snabbare)
!python scripts/evaluate_onnx.py --model ./models_eval/model.onnx --fakedata --limit 32


In [ ]:
# Visa utvärderingsresultat
import os

if os.path.exists("./reports/eval_summary.txt"):
    with open("./reports/eval_summary.txt", "r") as f:
        print("📊 Utvärderingsresultat:")
        print(f.read())
else:
    print("❌ Utvärderingsrapport saknas")


In [ ]:
# Visa confusion matrix om den finns
import matplotlib.pyplot as plt
from PIL import Image

if os.path.exists("./reports/confusion_matrix.png"):
    print("📈 Confusion Matrix:")
    img = Image.open("./reports/confusion_matrix.png")
    plt.figure(figsize=(8, 6))
    plt.imshow(img)
    plt.axis('off')
    plt.title('Confusion Matrix')
    plt.show()
else:
    print("❌ Confusion matrix saknas")


## 🔍 Automatisk verifiering

**Verifiering** = automatiska checks som säkerställer att lektionen fungerar korrekt.

**Vad kontrolleras**:
- **Artefakter finns** - alla nödvändiga filer är skapade
- **Benchmark fungerar** - latens-data är giltig
- **Kvantisering fungerar** - kvantiserad modell är skapad
- **Utvärdering fungerar** - confusion matrix och accuracy är tillgänglig

**Resultat**: `progress/receipt.json` med PASS/FAIL status


In [ ]:
# Kör automatisk verifiering
print("🔍 Kör automatisk verifiering...")
!python verify.py


In [ ]:
# Analysera kvittot i detalj
import json

if os.path.exists("./progress/receipt.json"):
    with open("./progress/receipt.json", "r") as f:
        receipt = json.load(f)
    
    print("📋 Detaljerad kvitto-analys:")
    print(f"Status: {'✅ PASS' if receipt['pass'] else '❌ FAIL'}")
    print(f"Timestamp: {receipt['timestamp']}")
    
    print("\n🔍 Kontroller:")
    for check in receipt['checks']:
        status = "✅" if check['ok'] else "❌"
        print(f"  {status} {check['name']}: {check['reason']}")
    
    print("\n📊 Metrics:")
    if 'metrics' in receipt:
        for metric, value in receipt['metrics'].items():
            print(f"  {metric}: {value}")
    
    print("\n📁 Genererade filer:")
    if 'artifacts' in receipt:
        for artifact in receipt['artifacts']:
            print(f"  - {artifact}")
else:
    print("❌ Kvitto saknas")


## 🤔 Reflektionsfrågor

<details>
<summary>💭 Vilka mål verifieras av vår automatiska check?</summary>

**Svar**: Vår verifiering kontrollerar:
- **Teknisk funktionalitet** - alla steg körs utan fel
- **Artefakt-generering** - nödvändiga filer skapas
- **Data-integritet** - rapporter är giltiga och parseable
- **Pipeline-integration** - alla komponenter fungerar tillsammans

**Vad som INTE verifieras**:
- Accuracy-kvalitet (bara att utvärdering körs)
- Latens-mål (bara att benchmark körs)
- Produktionsredo (bara att pipeline fungerar)

</details>

<details>
<summary>💭 Vad saknas för "produktion"?</summary>

**Svar**: För produktion behöver vi:
- **Riktig data** - inte FakeData
- **Accuracy-mål** - specifika krav på precision/recall
- **Latens-mål** - SLA-krav på inference-tid
- **Robusthet** - hantering av edge cases och fel
- **Monitoring** - kontinuerlig övervakning av prestanda
- **A/B-testing** - jämförelse av olika modeller
- **Rollback** - möjlighet att gå tillbaka till tidigare version

</details>


## 🎯 Ditt eget experiment

**Uppgift**: Kör verifiering på olika modeller och jämför kvittona.

**Förslag**:
- Träna modeller med olika inställningar
- Kör verifiering på varje modell
- Jämför kvittona och se vilka som passerar/failar
- Analysera vilka checks som är mest kritiska

**Kod att modifiera**:
```python
# Träna olika modeller och kör verifiering
MODELS = [
    {"epochs": 1, "batch_size": 128, "name": "quick"},
    {"epochs": 3, "batch_size": 64, "name": "balanced"},
    {"epochs": 5, "batch_size": 32, "name": "thorough"}
]

for model_config in MODELS:
    # Träna modell
    # Kör verifiering
    # Analysera kvitto
```


In [ ]:
# TODO: Implementera ditt experiment här
# Träna olika modeller och jämför kvittona

MODELS = [
    {"epochs": 1, "batch_size": 128, "name": "quick"},
    {"epochs": 3, "batch_size": 64, "name": "balanced"},
    {"epochs": 5, "batch_size": 32, "name": "thorough"}
]

print("🧪 Mitt experiment: Jämför olika modeller")
for model_config in MODELS:
    print(f"  - {model_config['name']}: epochs={model_config['epochs']}, batch_size={model_config['batch_size']}")

# TODO: Implementera loop som tränar och verifierar varje modell


## 🎉 Sammanfattning

Du har nu lärt dig:
- Vad utvärdering är och varför det är viktigt
- Hur automatisk verifiering fungerar
- Vad som kontrolleras i kvittot
- Vad som saknas för produktion

**Du har genomfört hela PiEdge EduKit-lektionen!** 🎉

**Vad du nu kan**:
- Träna edge-ML modeller med PyTorch
- Exportera till ONNX för edge deployment
- Benchmarka latens och förstå prestanda
- Kvantisera modeller för snabbare inference
- Verifiera att hela pipeline fungerar

**Nästa steg**: Experimentera med riktig data, olika modeller, eller deployera på Raspberry Pi!

**Viktiga begrepp**:
- **Utvärdering**: Testa modellen på osett data
- **Confusion matrix**: Detaljerad breakdown av prestanda
- **Verifiering**: Automatiska checks för pipeline-funktionalitet
- **Kvitto**: Strukturerad rapport med PASS/FAIL status
